In [32]:
import numpy as np
import pandas as pd
from clean import manifest_clinical_merge
from collections import Counter

In [2]:
pd.set_option('display.max_columns', 100)

# TLDR: Predicting Low Risk/ Not Low Risk using Gene Expression

Low Risk = 1; Standard - High Risk = 0

## Important Takeaway:

### Lasso Logistic Regression fit on all features (~180 important features)

### RF Classification fit on all features (~1100 important features)

### XGBoost Classification fit on all features (~300 important features)

### 11 ensemble gene ids considered important by all three models**

'ENSG00000099998.16', 'ENSG00000124491.14', 'ENSG00000125430.7', 'ENSG00000130649.8', 'ENSG00000151468.10', 'ENSG00000159708.16', 'ENSG00000163106.9', 'ENSG00000198842.8', 'ENSG00000228172.4', 'ENSG00000260182.1', 'ENSG00000266733.4'

# Data Prep

In [3]:
manifest_df = pd.read_csv('../Manifest/GCD_TARGET_Data_Manifest_AML_NBL_WT_RT.csv')

wt_disc_df = pd.read_excel('../Clinical_Data/TARGET_WT_ClinicalData_Discovery_20160714_public.xlsx')
aml_disc_df = pd.read_excel('../Clinical_Data/TARGET_AML_ClinicalData_20160714.xlsx')
nbl_disc_df = pd.read_excel('../Clinical_Data/TARGET_NBL_ClinicalData_20151124.xlsx')

WT_df = manifest_clinical_merge(manifest_df, wt_disc_df, 'TARGET-WT')
AML_df = manifest_clinical_merge(manifest_df, aml_disc_df, 'TARGET-AML')
NBL_df = manifest_clinical_merge(manifest_df, nbl_disc_df, 'TARGET-NBL')

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/clean.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  target_df['TARGET USI'] = target_df.loc[:, 'entity_submitter_id'].apply(lambda x: x[:16])


In [4]:
from clean import assay_transpose

In [353]:
assay_df = pd.read_csv('../Expn_Data/TARGET_NBL_AML_RT_WT_TMMCPM_log2_Norm_Counts.csv.zip')
assay_t_df = assay_transpose(assay_df)

In [6]:
from clean import assay_clinical_merge

In [7]:
AML_genes = assay_clinical_merge(assay_t_df, AML_df)
WT_genes = assay_clinical_merge(assay_t_df, WT_df)
NBL_genes = assay_clinical_merge(assay_t_df, NBL_df)

# Modeling on AML_genes

## Classification - predict low_risk / not_low_risk in AML patients

1) Variance comparison: Subset of patients under 03 09A classifcation - choosing 1000 genes of highest/lowest variance 
between labeled low_risk, not_low_risk group

2) Model performance when randomly sampling genes

3) Lasso logistic regression

4) Tree modeling with features considered important by Lasso

5) Modeling on all features with RF and XGBoost

In [46]:
from model_comp import data_prep_columns, model_prep, model_comp, model_prep_loc, xgboost_tuner, create_model_df
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# 1. Variance Comparison

### High Variance

In [215]:
df, a = data_prep_columns(AML_genes, 'Max')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, a)
xg1, rf1 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.28771180389607903
Gradient Boost Log Loss 0.36210740308746076
Random Forest Log Loss 0.3262005795045274
XGBoost Log Loss 0.22184269636164647


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [216]:
df.head()

,TARGET USI,Diagnostic ID,Gender,Race,Ethnicity,Age at Diagnosis in Days,First Event,Event Free Survival Time in Days,Vital Status,Overall Survival Time in Days,Year of Diagnosis,Year of Last Follow Up,Protocol,WBC at Diagnosis,Bone marrow leukemic blast percentage (%),Peripheral blasts (%),CNS disease,Chloroma,FAB Category,t(6;9),t(8;21),t(3;5)(q25;q34),t(6;11)(q27;q23),t(9;11)(p22;q23),t(10;11)(p11.2;q23),t(11:19)(q23:p13.1),inv(16),del5q,del7q,del9q,monosomy 5,monosomy 7,trisomy 8,trisomy 21,MLL,Minus Y,Minus X,Cytogenetic Code Other,Cytogenetic Complexity,Primary Cytogenetic Code,ISCN,FLT3/ITD positive?,FLT3/ITD allelic ratio,FLT3 PM,NPM mutation,CEBPA mutation,WT1 mutation,c-Kit Mutation Exon 8,c-Kit Mutation Exon 17,MRD at end of course 1,...,ENSG00000280913.1,ENSG00000280916.1,ENSG00000280927.1,ENSG00000280936.1,ENSG00000280953.1,ENSG00000280987.1,ENSG00000281005.1,ENSG00000281021.1,ENSG00000281026.1,ENSG00000281100.1,ENSG00000281103.1,ENSG00000281106.1,ENSG00000281128.1,ENSG00000281131.1,ENSG00000281183.1,ENSG00000281189.1,ENSG00000281195.1,ENSG00000281205.1,ENSG00000281207.1,ENSG00000281332.1,ENSG00000281344.1,ENSG00000281357.1,ENSG00000281358.1,ENSG00000281376.1,ENSG00000281383.1,ENSG00000281392.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281406.1,ENSG00000281433.1,ENSG00000281445.1,ENSG00000281468.1,ENSG00000281490.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281538.1,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1,__no_feature,__ambiguous,__alignment_not_unique,Low Risk
3,TARGET-20-PADZCG,09A,Female,Unknown,Not Hispanic or Latino,5325,Relapse,314,Dead,536,1996,1998,CCG-2961,29.3,58,5,No,No,M4,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,0,Normal,"46,XX[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-2.42284,0.314777,1.74517,-6.92714,-6.12136,1.40683,0.995511,-0.223406,2.2914,-0.613341,3.9011,1.61224,-4.6821,-6.92714,0.8846,-2.4712,0.732776,0.678398,0.391151,1.82242,1.4786,1.37983,0.894175,-1.11056,-0.330849,0.0915962,1.95643,-0.547052,-6.92714,-1.45498,1.12314,-2.93057,2.9388,1.76878,0.580973,-0.832714,-1.78095,0.132623,-0.682824,5.46561,-0.330849,-1.84323,-0.801464,-0.364707,2.1646,-0.152689,17.1243,14.6061,17.8527,0
5,TARGET-20-PAEAKL,09A,Female,Black or African American,Not Hispanic or Latino,5218,Relapse,134,Dead,163,1997,1997,CCG-2961,96.1,80,81,Yes,No,M5,No,No,No,No,Yes,No,Yes,No,No,No,No,No,No,No,No,Yes,No,No,,1,MLL,"46,XX,t(11:19)(q23:p13.1)[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-5.80459,-6.92714,1.03526,-6.92714,-6.92714,0.113774,1.32433,-4.41384,0.664587,-6.92714,1.21435,-6.92714,-6.92714,-5.80459,-0.0939682,-2.53384,-1.48807,-4.14335,-1.05214,0.929309,0.340124,0.95113,1.78753,-0.202203,6.32096,-0.544956,1.04881,0.496696,-6.92714,-2.79931,-0.914236,-1.94482,1.12118,3.16766,-2.61702,-1.89207,-3.1249,1.05554,-1.94482,3.83862,-2.17756,-0.717812,-0.88817,1.75482,0.456577,0.965496,16.7942,15.3635,19.6702,0
9,TARGET-20-PAEERJ,09A,Female,White,Not Hispanic or Latino,5082,Relapse,259,Dead,373,1997,1998,CCG-2961,71.9,60,41,No,No,M4,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,1,Other,"46,XX,t(3:5)(q23:q33)[19]/46,XX[1]",Yes,0.55,No,No,No,Yes,Not done,Not done,Unknown,...,-3.82289,-3.00939,2.17987,-6.92714,-6.92714,1.94937,-1.00327,0.722174,3.02575,-3.82289,4.43513,-3.49809,-6.92714,-6.92714,1.32028,-4.50961,1.67087,-2.35434,-0.680935,1.47645,1.32543,0.512625,0.884345,-0.488087,-1.00327,-1.25643,2.91237,-0.209853,-6.92714,-1.35175,-0.0681887,-2.81572,-0.0817453,0.557041,0.30953,-2.56667,-1.85807,-0.470147,1.27314,6.07709,0.768241,-0.600677,-1.56366,-0.506252,2.3282,-2.90931,16.916,14.5652,17.571,0
10,TARGET-20-PAEFGR,09A,Female,White,Not Hispanic or Latino,549,Relapse,497,Alive,4022,1997,2008,CCG-2961,51.2,75,41,No,No,M2,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,,1,inv(16),"46,XX,inv(16)(p13q22)[21]/46,XX[1]",No,NaN,No,No,No

In [218]:
xg1_important = X.columns[np.array(xg1.feature_importances_) > 0]
rf1_important = X.columns[np.array(rf1.feature_importances_) > 0]

In [219]:
print(len(xg1_important))
len(rf1_important)

205


697

### country frequency of genes being labeled 'important' by RF or XGboost

In [35]:
# xg_gene_counter = Counter()
# rf_gene_counter = Counter()

In [36]:
# for gene in xg1_important:
#     xg_gene_counter[gene] +=1
# for gene in rf1_important:
#     rf_gene_counter[gene] +=1

### 1b. low variance genes between two labeled groups

In [220]:
df, a = data_prep_columns(AML_genes, 'Min')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, a)
xg2, rf2 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.4236865689681798
Gradient Boost Log Loss 0.49256451159202913
Random Forest Log Loss 0.48223471620602953
XGBoost Log Loss 0.3429010650394736


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [221]:
xg2_important = X.columns[np.array(xg2.feature_importances_) > 0]
rf2_important = X.columns[np.array(rf2.feature_importances_) > 0]

In [223]:
print(len(xg2_important))
len(rf2_important)

208


766

In [222]:
# for gene in xg2_important:
#     xg_gene_counter[gene] +=1
# for gene in rf2_important:
#     rf_gene_counter[gene] +=1

# 2.random gene selection

- random_cols_ are **column index numbers** chosen at random

- does not perform as well as the variation selected columns

- use **model_prep** instead of **model_prep_loc** as random_columns_ is a randomly generated list of numbers 
used to select columns

In [230]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]

In [231]:
# random_cols_ are column index numbers

In [232]:
len(random_cols_)

1914

In [233]:
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, random_cols_)
xg_random, rf_random = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.39491046043536376
Gradient Boost Log Loss 0.4643323954434488
Random Forest Log Loss 0.4528452946267843
XGBoost Log Loss 0.3226523800073443


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [228]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, random_cols_)
xg_random2, rf_random2 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3857378566561362
Gradient Boost Log Loss 0.450060344474042
Random Forest Log Loss 0.42749734747068563
XGBoost Log Loss 0.3699347494119728


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [234]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep(df, random_cols_)
xg_random3, rf_rando3 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3772384943194
Gradient Boost Log Loss 0.504297547416616
Random Forest Log Loss 0.4216150371839307
XGBoost Log Loss 0.3159291592955187


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


# 3. Lasso Logistic Regression using all columns

- lasso reduces to 3212 genes instead of 20,000+

**see bottom to see fully converged lasso**

In [69]:
from sklearn.linear_model import LogisticRegression

In [235]:
lasso_columns = df.iloc[:, 84:-4].columns
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep_loc(df, lasso_columns)
log_model = LogisticRegression(penalty='l1', solver='saga')
log_model.fit(X_train, y_train)
y_pred = log_model.predict_proba(X_test)
log_loss(y_test, y_pred)

/Users/david/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.21263654943160015

In [236]:
log_coefs = np.array(log_model.coef_)
mask = log_coefs > 0
important_lasso_genes = lasso_columns[mask[0]]

In [237]:
len(important_lasso_genes)

3110

In [ ]:
# for gene in important_lasso_genes:
#     xg_gene_counter[gene] +=1
#     rf_gene_counter[gene] +=1

### 4. Trees Performance Using Lasso Genes

In [238]:
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep_loc(df, important_lasso_genes)
xg_lass, rf_lass = model_comp(X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.38889814893691244
Gradient Boost Log Loss 0.4288557996806494
Random Forest Log Loss 0.40714678601797455
XGBoost Log Loss 0.38903383744528164


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [239]:
xg_lass_important = X.columns[np.array(xg_lass.feature_importances_) > 0]
rf_lass_important = X.columns[np.array(rf_lass.feature_importances_) > 0]

In [86]:
xgboost_tuner(X_train, X_test, y_train, y_test, [4, 6, 8, 10, 12])

XGB n_estimators: 400 log loss 0.4201527334044914
XGB n_estimators: 600 log loss 0.3856683401894328
XGB n_estimators: 800 log loss 0.3854842494317406
XGB n_estimators: 1000 log loss 0.3859489940459261
XGB n_estimators: 1200 log loss 0.3860728672479053


# 5. Feature Importance When Running Tree Over All Features

In [344]:
all_columns = df.iloc[:, 84:-4].columns
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, X = model_prep_loc(df, all_columns)
xg_all, rf_all = model_comp(X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3417890185477423
Gradient Boost Log Loss 0.4407397864037863
Random Forest Log Loss 0.39492259532890717
XGBoost Log Loss 0.25292958437490304


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [345]:
xg_all_important = X.columns[np.array(xg_all.feature_importances_) > 0]
rf_all_important = X.columns[np.array(rf_all.feature_importances_) > 0]

In [346]:
print(len(xg_all_important))
len(rf_all_important)

332


1147

In [348]:
# holdout.iloc[:, :-1].head()

In [349]:
# holdout.iloc[:, :-1].info()

### Performance with holdout set

In [350]:
p_holdout_pred =  xg_all.predict_proba(holdout.iloc[:, :-1].values)

In [351]:
log_loss(holdout.iloc[:, -1], p_holdout_pred)

0.18713662400841713

# Comparing the features that the models think are important

### Lasso vs. XGBoost vs. RF (on all columns)

In [243]:
print(len(important_lasso_genes))
print(len(xg_all_important))
len(rf_all_important)

3110
332


1147

In [244]:
set_lasso = set(important_lasso_genes)
set_xg = set(xg_all_important)
set_rf = set(rf_all_important)

In [245]:
print(len(set(important_lasso_genes)))
print(len(set(xg_all_important)))
len(set(rf_all_important))

3110
332


1147

Genes deemed important by both **xg and rf**

In [246]:
xg_rf_intersection = set_xg.intersection(set_rf)
len(xg_rf_intersection)

48

Genes deemed important by both **lasso and rf**

In [247]:
lasso_rf_intersection = set_lasso.intersection(set_rf)
len(lasso_rf_intersection)

290

Genes deemed important by both **lasso and xg**

In [248]:
lasso_xg_intersection = set_lasso.intersection(set_xg)
len(lasso_xg_intersection)

75

Intersection of ** lasso, xg and rf**

In [249]:
three_way = set.intersection(set_lasso, set_xg, set_rf)
len(three_way)

11

In [250]:
three_way

{'ENSG00000099998.16',
 'ENSG00000124491.14',
 'ENSG00000125430.7',
 'ENSG00000130649.8',
 'ENSG00000151468.10',
 'ENSG00000159708.16',
 'ENSG00000163106.9',
 'ENSG00000198842.8',
 'ENSG00000228172.4',
 'ENSG00000260182.1',
 'ENSG00000266733.4'}

### THESE ARE OUR MOST IMPORTANT GENE ENSEMBLES

'ENSG00000099998.16',
 'ENSG00000124491.14',
 'ENSG00000125430.7',
 'ENSG00000130649.8',
 'ENSG00000151468.10',
 'ENSG00000159708.16',
 'ENSG00000163106.9',
 'ENSG00000198842.8',
 'ENSG00000228172.4',
 'ENSG00000260182.1',
 'ENSG00000266733.4'

# Making a table of genes 

In [106]:
all_genes = set.union(set_lasso, set_xg, set_rf)
len(all_genes)

4285

In [ ]:
# test1 = ['a', 'b', 'c']
# test2 = ['d', 'c', 'e']
# test_list ['a', 'b', 'c']
# test_dict = {}
# for letter in test_list:
#     if item in 



In [107]:
# gene_dict = {}
# for gene in all_genes:
#     if gene in three_way:
#         gene_dict[gene] = 'RF, XGBoost, Lasso'
#     elif gene in lasso_xg_intersection:
#         gene_dict[gene] = 'XGBoost, Lasso'
#     elif gene in lasso_rf_intersection:
#         gene_dict[gene] = 'RF, Lasso'
        

SETS IM WORKING WITH:


-  three_way (union of all 11)

- lasso_xg_intersection, xg_rf_intersection, lasso_rf_intersection

- set_lasso, set_xg, set_rf (individual)

In [115]:
gene_dict = {}
for gene in all_genes:
    if gene not in gene_dict:
        gene_dict[gene] = []
        if gene in set_lasso:
            gene_dict[gene].append('lasso')
        if gene in set_xg:
            gene_dict[gene].append('xgboost')
        if gene in set_rf:
            gene_dict[gene].append('random forest')
    else:
        if gene in set_lasso:
            gene_dict[gene].append('lasso')
        if gene in set_xg:
            gene_dict[gene].append('xgboost')
        if gene in set_rf:
            gene_dict[gene].append('random forest')

In [180]:
gene_dict = {}
for gene in all_genes:
    if gene not in gene_dict:
        gene_dict[gene] = ''
        if gene in set_lasso:
            gene_dict[gene] += 'lasso '
        if gene in set_xg:
            gene_dict[gene] += 'xgb '
        if gene in set_rf:
            gene_dict[gene] += 'rf '
    else:
        if gene in set_lasso:
            gene_dict[gene] += ' lasso '
        if gene in set_xg:
            gene_dict[gene] += ' xgb '
        if gene in set_rf:
            gene_dict[gene] += ' rf '

In [181]:
len(gene_dict)

4285

In [182]:
gene_dict['ENSG00000259321.1']

'lasso rf '

In [183]:
d = list(gene_dict.values())

In [184]:
important_genes = pd.DataFrame(data = d, index=gene_dict.keys(), columns = ['Model'])

In [288]:
important_genes.head(5)

,Model
ENSG00000173597.7,lasso
ENSG00000143815.13,rf
ENSG00000110583.11,rf
ENSG00000235776.2,rf
ENSG00000104213.11,lasso


In [ ]:
# if gene in set_lasso:
#             gene_dict[gene] += ' lasso '
#         if gene in set_xg:
#             gene_dict[gene] += ' xgb '
#         if gene in set_rf:
#             gene_dict[gene] += ' rf '

In [186]:
lasso_dict = {}
for gene in all_genes:
    if gene in set_lasso:
        lasso_dict[gene] = 1
    else:
        lasso_dict[gene] = 0

rf_dict = {}
for gene in all_genes:
    if gene in set_rf:
        rf_dict[gene] = 1
    else:
        rf_dict[gene] = 0
        
xgb_dict = {}
for gene in all_genes:
    if gene in set_xg:
        xgb_dict[gene] = 1
    else:
        xgb_dict[gene] = 0

In [190]:
a = list(lasso_dict.values())
b = list(rf_dict.values())
c = list(xgb_dict.values())

In [210]:
pd.DataFrame.from_dict([lasso_dict, rf_dict, xgb_dict])

,ENSG00000000003.13,ENSG00000002586.16,ENSG00000002726.18,ENSG00000002933.6,ENSG00000003056.6,ENSG00000003137.7,ENSG00000003249.12,ENSG00000003436.13,ENSG00000004660.13,ENSG00000004939.12,ENSG00000005073.5,ENSG00000005102.11,ENSG00000005108.14,ENSG00000005194.13,ENSG00000005243.8,ENSG00000005381.7,ENSG00000005471.14,ENSG00000005513.9,ENSG00000005884.16,ENSG00000005893.14,ENSG00000006025.10,ENSG00000006042.10,ENSG00000006047.11,ENSG00000006118.13,ENSG00000006283.16,ENSG00000006459.9,ENSG00000006534.14,ENSG00000006611.14,ENSG00000006638.10,ENSG00000006747.13,ENSG00000007001.11,ENSG00000007038.9,ENSG00000007129.16,ENSG00000007237.17,ENSG00000007314.10,ENSG00000007350.15,ENSG00000007402.10,ENSG00000007516.12,ENSG00000007866.17,ENSG00000007944.13,ENSG00000008056.11,ENSG00000008283.14,ENSG00000008311.13,ENSG00000008394.11,ENSG00000008516.15,ENSG00000008517.15,ENSG00000009790.13,ENSG00000010319.5,ENSG00000010704.17,ENSG00000010818.7,...,ENSG00000279978.1,ENSG00000279989.1,ENSG00000280008.1,ENSG00000280011.1,ENSG00000280022.1,ENSG00000280065.1,ENSG00000280079.1,ENSG00000280081.2,ENSG00000280099.1,ENSG00000280123.1,ENSG00000280129.1,ENSG00000280132.1,ENSG00000280153.1,ENSG00000280157.1,ENSG00000280184.1,ENSG00000280187.1,ENSG00000280194.1,ENSG00000280195.1,ENSG00000280211.1,ENSG00000280213.1,ENSG00000280216.1,ENSG00000280237.1,ENSG00000280254.1,ENSG00000280287.1,ENSG00000280303.2,ENSG00000280332.1,ENSG00000280351.2,ENSG00000280401.1,ENSG00000280550.1,ENSG00000280560.1,ENSG00000280594.1,ENSG00000280614.1,ENSG00000280623.1,ENSG00000280649.1,ENSG00000280670.1,ENSG00000280721.1,ENSG00000280777.1,ENSG00000280798.1,ENSG00000280953.1,ENSG00000281106.1,ENSG00000281207.1,ENSG00000281344.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281433.1,ENSG00000281468.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281571.1,ENSG00000281741.1
0,0,1,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,1,1,0,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,...,0,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1
1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


# DIRECTIONALITY

looking at 11 genes deemed important by all three models

In [262]:
def find_indices(lst, lst2):
    return [i for i, elem in enumerate(lst) if elem in lst2]

In [276]:
X.head()

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,ENSG00000001460.16,ENSG00000001461.15,ENSG00000001497.15,ENSG00000001561.6,ENSG00000001617.10,ENSG00000001626.13,ENSG00000001629.8,ENSG00000001630.14,ENSG00000001631.13,ENSG00000002016.15,ENSG00000002330.12,ENSG00000002549.11,ENSG00000002586.16,ENSG00000002587.8,ENSG00000002726.18,ENSG00000002746.13,ENSG00000002822.14,ENSG00000002834.16,ENSG00000002919.13,ENSG00000002933.6,ENSG00000003056.6,ENSG00000003096.12,ENSG00000003137.7,ENSG00000003147.16,ENSG00000003249.12,ENSG00000003393.13,ENSG00000003400.13,ENSG00000003402.18,ENSG00000003436.13,ENSG00000003509.14,ENSG00000003756.15,ENSG00000003987.12,ENSG00000003989.15,ENSG00000004059.9,ENSG00000004139.12,ENSG00000004142.10,ENSG00000004399.11,ENSG00000004455.15,ENSG00000004468.11,ENSG00000004478.7,...,ENSG00000280789.1,ENSG00000280798.1,ENSG00000280828.1,ENSG00000280832.1,ENSG00000280913.1,ENSG00000280916.1,ENSG00000280927.1,ENSG00000280936.1,ENSG00000280953.1,ENSG00000280987.1,ENSG00000281005.1,ENSG00000281021.1,ENSG00000281026.1,ENSG00000281100.1,ENSG00000281103.1,ENSG00000281106.1,ENSG00000281128.1,ENSG00000281131.1,ENSG00000281183.1,ENSG00000281189.1,ENSG00000281195.1,ENSG00000281205.1,ENSG00000281207.1,ENSG00000281332.1,ENSG00000281344.1,ENSG00000281357.1,ENSG00000281358.1,ENSG00000281376.1,ENSG00000281383.1,ENSG00000281392.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281406.1,ENSG00000281433.1,ENSG00000281445.1,ENSG00000281468.1,ENSG00000281490.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281538.1,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1
5,-2.0565,-4.74701,4.01127,1.7086,2.45773,8.33617,-0.390612,5.86531,2.81586,4.3824,2.7385,2.93014,5.38245,0.610766,-0.00724986,-6.92715,2.7025,0.592369,2.33861,2.51256,4.33478,5.21782,8.32612,-2.53384,1.66084,-1.48807,4.79562,6.58635,4.58207,3.27681,6.57761,0.929309,-5.18099,1.85467,-1.56858,1.28478,3.38364,5.60581,-0.251195,1.83133,5.2888,-0.740952,-2.38062,6.86317,2.74684,6.33174,7.06771,7.84417,5.86555,6.0705,...,2.52717,1.46242,1.00775,3.60854,-5.80459,-6.92714,1.03526,-6.92714,-6.92714,0.113774,1.32433,-4.41384,0.664587,-6.92714,1.21435,-6.92714,-6.92714,-5.80459,-0.0939682,-2.53384,-1.48807,-4.14335,-1.05214,0.929309,0.340124,0.95113,1.78753,-0.202203,6.32096,-0.544956,1.04881,0.496696,-6.92714,-2.79931,-0.914236,-1.94482,1.12118,3.16766,-2.61702,-1.89207,-3.1249,1.05554,-1.94482,3.83862,-2.17756,-0.717812,-0.88817,1.75482,0.456577,0.965496
9,-0.600677,-6.92715,4.33729,3.70407,1.45783,5.80541,3.08307,4.70263,5.93286,3.68418,1.49483,3.81156,4.45815,5.90521,2.60909,-6.92715,3.98982,0.267381,3.63427,1.96923,3.15996,5.24463,8.50432,-1.56366,-0.0547583,-5.86665,2.9938,7.11946,4.06171,-0.879721,5.81253,0.494469,-4.50961,3.90215,-0.239935,4.13606,4.74356,6.5404,5.79849,3.05547,6.71508,2.10406,-3.35954,5.6307,2.3536,5.15138,6.21046,5.87504,5.85272,2.3282,...,0.370532,2.74585,2.34096,-1.0293,-3.82289,-3.00939,2.17987,-6.92714,-6.92714,1.94937,-1.00327,0.722174,3.02575,-3.82289,4.43513,-3.49809,-6.92714,-6.92714,1.32028,-4.50961,1.67087,-2.35434,-0.680935,1.47645,1.32543,0.512625,0.884345,-0.488087,-1.00327,-1.25643,2.91237,-0.209853,-6.92714,-1.35175,-0.0681887,-2.81572,-0.0817453,0.557041,0.30953,-2.56667,-1.85807,-0.470147,1.27314,6.07709,0.768241,-0.600677,-1.56366,-0.506252,2.3282,-2.90931
14,-1.44812,-5.93369,4.49801,1.87355,1.67079,6.36209,-2.93944,5.49176,4.04131,5.00884,-0.132792,2.92239,4.47019,5.27709,1.57547,-6.92715,4.28916,1.76021,3.41949,2.13122,3.20822,5.70585,8.3289,-1.54763,0.659716,-5.93369,3.81296,6.66139,4.0911,4.97972,6.08954,2.85579,-4.13103,0.0710648,-0.171959,4.14322,4.03631,6.40583,3.59599,2.86549,6.20551,0.667133,0.984602,5.39652,1.0023,5.3535,5.37063,6.66,5.00701,3.5516,...,0.104229,1.92395,1.3

In [284]:
direct_mask = find_indices(X.columns, three_way)

In [285]:
direct_mask

[2046, 5133, 5227, 5805, 8576, 9431, 9905, 15011, 16650, 19201, 19735]

In [286]:
log_model.coef_[0][direct_mask]

array([0.00293309, 0.00267322, 0.0011049 , 0.00313885, 0.00027107,
       0.0018612 , 0.00510481, 0.00475758, 0.00019398, 0.0070231 ,
       0.00383056])

# Make lasso converge - takeaways:

- second lasso took 3212 features --> 174

- intersection between new lasso (174) and XGboost (332) --> 15 in common

- of those 15 - 6 were in the original 11, 9 new

In [291]:
lasso_columns = df.iloc[:, 84:-4].columns
df, __ = data_prep_columns(AML_genes, 'neither')
X_train2, X_test2, y_train2, y_test2, holdout, X = model_prep_loc(df, lasso_columns)
log_model2 = LogisticRegression(penalty='l1', solver='saga', max_iter=10000)
log_model2.fit(X_train2, y_train2)
y_pred2 = log_model2.predict_proba(X_test2)
log_loss(y_test2, y_pred2)

0.18823188523395454

In [292]:
log_coefs2 = np.array(log_model2.coef_)
mask2 = log_coefs2 > 0
important_lasso_genes2 = lasso_columns[mask2[0]]
len(important_lasso_genes2)

174

In [314]:
holdout.head()

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,ENSG00000001460.16,ENSG00000001461.15,ENSG00000001497.15,ENSG00000001561.6,ENSG00000001617.10,ENSG00000001626.13,ENSG00000001629.8,ENSG00000001630.14,ENSG00000001631.13,ENSG00000002016.15,ENSG00000002330.12,ENSG00000002549.11,ENSG00000002586.16,ENSG00000002587.8,ENSG00000002726.18,ENSG00000002746.13,ENSG00000002822.14,ENSG00000002834.16,ENSG00000002919.13,ENSG00000002933.6,ENSG00000003056.6,ENSG00000003096.12,ENSG00000003137.7,ENSG00000003147.16,ENSG00000003249.12,ENSG00000003393.13,ENSG00000003400.13,ENSG00000003402.18,ENSG00000003436.13,ENSG00000003509.14,ENSG00000003756.15,ENSG00000003987.12,ENSG00000003989.15,ENSG00000004059.9,ENSG00000004139.12,ENSG00000004142.10,ENSG00000004399.11,ENSG00000004455.15,ENSG00000004468.11,ENSG00000004478.7,...,ENSG00000280798.1,ENSG00000280828.1,ENSG00000280832.1,ENSG00000280913.1,ENSG00000280916.1,ENSG00000280927.1,ENSG00000280936.1,ENSG00000280953.1,ENSG00000280987.1,ENSG00000281005.1,ENSG00000281021.1,ENSG00000281026.1,ENSG00000281100.1,ENSG00000281103.1,ENSG00000281106.1,ENSG00000281128.1,ENSG00000281131.1,ENSG00000281183.1,ENSG00000281189.1,ENSG00000281195.1,ENSG00000281205.1,ENSG00000281207.1,ENSG00000281332.1,ENSG00000281344.1,ENSG00000281357.1,ENSG00000281358.1,ENSG00000281376.1,ENSG00000281383.1,ENSG00000281392.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281406.1,ENSG00000281433.1,ENSG00000281445.1,ENSG00000281468.1,ENSG00000281490.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281538.1,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1,label
139,-1.44173,-6.92715,5.38677,2.49248,2.93061,5.05417,-1.15394,4.67788,4.36731,5.16476,1.21325,3.7806,4.44626,4.45698,0.797512,-5.65638,4.30682,1.524,4.15432,2.93061,3.08466,5.09395,6.13487,-1.53572,1.34437,-4.99139,3.4134,6.26471,3.51799,-1.74436,5.17268,-1.8016,-6.92715,5.15445,1.311,4.10065,4.38586,7.53537,3.91214,3.77694,6.59434,2.18145,-3.68266,5.04253,1.72485,4.3754,5.85451,6.30658,5.86403,4.30173,...,2.05817,3.01177,2.26048,-6.92714,-4.53794,1.05297,-6.92714,-1.44173,1.47061,0.787843,1.67326,2.17776,0.00959647,2.33872,-0.978675,-6.92714,-6.92714,2.00492,-1.8016,1.49458,-4.53794,0.881736,1.85092,1.31774,1.34437,1.71977,-0.504684,2.47149,0.995319,1.68373,0.335818,-6.92714,-1.48796,1.5978,-2.36605,1.56986,2.30861,0.253909,-1.48796,-0.793094,0.872617,-1.8612,5.62432,1.05297,-2.36605,-0.481113,-1.58512,2.41608,0.0424957,0
12,0.104784,-6.92715,4.81266,2.55302,0.438541,3.83624,1.24481,3.59023,4.07956,3.15866,-0.179674,2.51379,4.11495,5.30347,2.16562,-6.92715,2.90603,-0.0176145,3.56699,2.66071,3.71078,4.40671,8.26942,-1.88564,-2.93597,-4.0422,2.30493,7.25928,3.83977,-2.93597,5.44518,-1.16631,-3.67684,1.60152,1.62626,4.15375,5.26723,6.5717,5.57686,3.67566,6.71504,1.96692,-6.92715,5.30985,2.65671,4.90231,6.03044,4.78271,2.48702,2.5954,...,3.49942,2.35512,-1.28429,-3.67684,-4.0422,1.47104,-6.92714,-6.92714,4.29801,-1.79528,1.0018,1.82454,-2.75471,4.68013,-2.75471,-6.92714,-6.92714,1.37701,-5.28148,0.260374,-3.38555,-1.41279,0.301911,-0.463034,2.06064,0.693889,-0.0433916,1.3376,-1.16631,5.35495,0.72476,-6.92714,-1.55386,1.7364,-2.75471,-1.4816,0.419797,0.128065,-2.5937,-1.79528,1.22335,0.362058,6.54411,0.646308,-0.395081,-1.28429,-0.907716,2.45055,-1.55386,0
38,-5.61165,-6.92715,3.90408,2.64333,2.25689,8.4203,-2.2936,5.92398,3.23892,2.79599,3.40475,3.68965,5.34347,2.83362,4.78065,-6.92715,1.56994,-0.406389,1.02794,0.929438,6.28928,5.00335,9.33731,-3.07916,5.74993,-4.12899,6.0315,6.58457,4.96882,2.82617,6.64543,0.196616,-5.61165,1.15227,-0.777865,1.917,2.86787,5.85063,0.833652,1.54596,4.26056,-0.360345,-3.84916,7.05257,0.463057,6.8902,6.24552,8.31644,5.87098,5.23423,...,0.500967,1.72214,3.54745,-4.12899,-6.92714,-0.230424,-6.92714,-6.9

In [316]:
y_holdout_pred =  log_model2.predict_proba(holdout.iloc[:, :-1])
log_loss(holdout.iloc[:, -1], y_holdout_pred)

0.17037272553094038

In [312]:
pd.DataFrame(data = important_lasso_genes2).to_csv('lasso_genes_178.csv')

In [310]:
important_lasso_genes2

Index(['ENSG00000005108.14', 'ENSG00000005513.9', 'ENSG00000019169.10',
       'ENSG00000067798.12', 'ENSG00000069535.13', 'ENSG00000072694.17',
       'ENSG00000079102.15', 'ENSG00000080493.12', 'ENSG00000087245.11',
       'ENSG00000095917.12',
       ...
       'ENSG00000272217.1', 'ENSG00000272342.1', 'ENSG00000272973.1',
       'ENSG00000273355.1', 'ENSG00000273449.1', 'ENSG00000273628.1',
       'ENSG00000274191.1', 'ENSG00000275769.1', 'ENSG00000279717.1',
       'ENSG00000280081.2'],
      dtype='object', length=174)

In [294]:
# set_xg

In [296]:
lasso_set2 = set(important_lasso_genes2)

In [300]:
xgb_lasso_set2 = lasso_set2.intersection(set_xg)
len(xgb_lasso_set2)

15

In [301]:
three_way

{'ENSG00000099998.16',
 'ENSG00000124491.14',
 'ENSG00000125430.7',
 'ENSG00000130649.8',
 'ENSG00000151468.10',
 'ENSG00000159708.16',
 'ENSG00000163106.9',
 'ENSG00000198842.8',
 'ENSG00000228172.4',
 'ENSG00000260182.1',
 'ENSG00000266733.4'}

In [302]:
xgb_lasso_set2

{'ENSG00000095917.12',
 'ENSG00000111341.8',
 'ENSG00000115112.7',
 'ENSG00000124491.14',
 'ENSG00000130649.8',
 'ENSG00000163106.9',
 'ENSG00000179639.9',
 'ENSG00000182700.4',
 'ENSG00000185345.17',
 'ENSG00000198842.8',
 'ENSG00000255882.1',
 'ENSG00000257207.5',
 'ENSG00000260182.1',
 'ENSG00000266733.4',
 'ENSG00000267453.5'}

In [303]:
xgb_lasso_set2.intersection(three_way)

{'ENSG00000124491.14',
 'ENSG00000130649.8',
 'ENSG00000163106.9',
 'ENSG00000198842.8',
 'ENSG00000260182.1',
 'ENSG00000266733.4'}

In [304]:
xgb_lasso_set2.difference(three_way)

{'ENSG00000095917.12',
 'ENSG00000111341.8',
 'ENSG00000115112.7',
 'ENSG00000179639.9',
 'ENSG00000182700.4',
 'ENSG00000185345.17',
 'ENSG00000255882.1',
 'ENSG00000257207.5',
 'ENSG00000267453.5'}